In [164]:
# Dependencies
import requests
import json
import pandas as pd
from sqlalchemy import create_engine

In [165]:
df = pd.read_csv("Resources/austin_crime.csv")
df
df.dropna(subset=['zipcode', 'year','description','primary_type'])
#city_data = pd.read_csv("Resources/austin_crime.csv", index_col='zipcode', names=['year'])
df.head(2)


,address,census_tract,clearance_date,clearance_status,council_district_code,description,district,latitude,location,location_description,longitude,primary_type,timestamp,unique_key,x_coordinate,y_coordinate,year,zipcode
0,2620 RIO GRANDE ST Austin TX 78705,6.03,2015-12-14 00:00:00,Not cleared,9.0,THEFT,B,NaN,NaN,2620 RIO GRANDE ST,NaN,Theft,2015-12-13 00:00:00,2.015505e+10,3113531.0,10079579.0,2015.0,78705.0
1,9500 S IH 35 SVRD SB Austin TX 78748,24.21,2015-09-14 00:00:00,Cleared by Arrest,5.0,THEFT BY SHOPLIFTING,F,NaN,NaN,9500 S IH 35 SVRD SB,NaN,Theft,2015-09-03 00:00:00,2.015246e+10,3099459.0,10032098.0,2015.0,78748.0


In [166]:
df
df=df.loc[:,'description':'zipcode']
df = df[pd.notnull(df['zipcode'])]
col=["description", "year", "zipcode"]
df=df.drop(['district','latitude','location','location_description','longitude','primary_type','timestamp','unique_key','x_coordinate','y_coordinate'],axis=1)
df.head(3)

,description,year,zipcode
0,THEFT,2015.0,78705.0
1,THEFT BY SHOPLIFTING,2015.0,78748.0
4,THEFT BY SHOPLIFTING,2014.0,78751.0


In [167]:
df['zipcode'] = df['zipcode'].map(lambda x: str(x)[:-2])
df['zipcode'].value_counts()


78753    6967
78741    5769
78758    4977
78704    4968
78745    4564
78701    4056
78723    3877
78744    3709
78702    3490
78748    2883
78759    2804
78705    2351
78752    2337
78757    2095
78751    1753
78749    1577
78746    1566
78703    1322
78731    1010
78727     993
78721     975
78724     972
78729     884
78613     791
78754     791
78750     756
78756     631
78722     622
78735     528
78617     502
78747     468
78717     438
78726     425
78719     312
78739     305
78660     244
78736     118
78725     106
78730      90
78742      65
78653      45
78728      19
78652      16
78712      15
78737       6
78732       4
78733       3
78610       2
Name: zipcode, dtype: int64

In [172]:
city_data = df.groupby('zipcode').count()
city_crime_data = city_data.drop(['description'],axis=1)
city_crime_data = city_crime_data.rename(columns={'year':'count'})

In [173]:
from sqlalchemy import create_engine, inspect
import pymysql
pymysql.install_as_MySQLdb()

In [174]:
connection = "root:Sasha_2008@127.0.0.1/austin_life_zip_db"
engine = create_engine(f'mysql://{connection}')
engine.table_names()


['crime_data', 'imr', 'parks', 'public_health']

In [175]:
city_crime_data
city_crime_data.to_sql(name='crime_data', con=engine, if_exists='append', index=True)